In [1]:
import cv2
import time
import mediapipe as mp
from pysinewave import SineWave

In [2]:
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
	min_detection_confidence=0.5,
	min_tracking_confidence=0.5
)

# Initializing the drawing utils for drawing landmarks on image
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [3]:
cap = cv2.VideoCapture(0)
with holistic_model as holistic:
    sinewave = SineWave(pitch = -36, pitch_per_second = 20)
    sinewave.play()
    sinewave.set_volume(0)
    while cap.isOpened():
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          continue
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Drawing Right hand Land Marks
        mp_drawing.draw_landmarks(
          image,
          results.right_hand_landmarks,
          mp_holistic.HAND_CONNECTIONS
        )
        
        # Drawing Left hand Land Marks
        mp_drawing.draw_landmarks(
          image,
          results.left_hand_landmarks,
          mp_holistic.HAND_CONNECTIONS
        )

        #65.4-2093hz
        #2027.6 difference
        #frequency = 440 * 2^((pitch-9)/12)
        h, w, _ = image.shape #h=720 w=1280
        #40px padding -> 40<h<680 (640), 40<w<1240 (1200)
        
        if results.right_hand_landmarks == None:
            sinewave.set_volume(volume=0)
        else: 
            right_pinky = results.right_hand_landmarks.landmark[20]
            right_pinky_x = int(right_pinky.x * w) # Convert landmark positions to pixel coordinates
            # print(f"Right Pinky Tip: {right_pinky_x}")
            # freq = max(((1240-right_pinky_x)*1.69)+65.4, 0)
            pitch = ((1280-right_pinky_x)*0.047)-36
            sinewave.set_pitch(pitch)
            # sinewave.set_frequency(freq) #each pixel diff. = 1.69 Hz
            # print(f"pitch= {pitch}")
        if results.left_hand_landmarks == None:
            sinewave.set_volume(volume=0)   
        else: 
            left_mid = results.left_hand_landmarks.landmark[12]
            left_mid_y = int(left_mid.y * h) # Convert landmark positions to pixel coordinates
            vol = max(((640-left_mid_y)/17.22)+30, 0)
            # print(f"Left mid: {left_mid_y}")
            sinewave.set_volume(volume=vol) #range = 30db to 75db / each db = 17.22 px
            # print(f"volume: {vol}")
        # Flip the image horizontally for mirrored display
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
          sinewave.stop()
          break

# When all the process is done
# Release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
